In [1]:
# ! mongod --dbpath /Volumes/Transcend/data/db --fork --logpath ~/Library/Logs/mongodb.log

In [2]:
from pymongo import MongoClient
import pandas as pd
import os

In [3]:
client = MongoClient()
db = client.birt

In [4]:
# Make sure we've got the thing hooked up right.
db.collection_names()

['migrations', 'birds', 'halunka:i18n']

In [5]:
birds = db.birds

So, now we have `birds`, which is a collection of the eBird sightings. The documentation for the `Collection()` class is [here](https://api.mongodb.org/python/current/api/pymongo/collection.html#pymongo.collection.Collection). `birds.find_one()` will get us the first record to take a look at the structure.

In [6]:
birds.find_one()

{'_id': 'abeillia_abeillei',
 'category': 'species',
 'family_name': 'Trochilidae (Hummingbirds)',
 'genus_name': 'Abeillia',
 'order_name': 'Apodiformes',
 'primary_com_name': 'Emerald-chinned_Hummingbird',
 'species_name': 'abeillei',
 'subfamily_name': None,
 'taxon_order': 9016.0}

In [7]:
migrations = db.migrations

Below, we examine a single `migrations` document. It includes... gosh, I think this data is pretty shittily organized. How can I index this? It doesn't have like, a "sightings" array. It has just, all the variables, and then the number of birds at that location.

So, for each bird, I should:

1. Find all documents with that species name.
2. Extract all covariates for those locations, and the "target", in `scikit-learn` terminology, is the number of birds.
3. Load all background covariates

In [8]:
migrations.find_one()

{'_id': 'S10000010',
 'agelaius_phoeniceus': 2,
 'baeolophus_bicolor': 4,
 'bailey_ecoregion': '-222J',
 'bcr': 12,
 'bucephala_clangula': 9,
 'cardinalis_cardinalis': 1,
 'caus_prec': 3,
 'caus_snow': 3,
 'caus_temp_avg': 2,
 'caus_temp_max': 1,
 'caus_temp_min': 2,
 'corvus_brachyrhynchos': 2,
 'count_type': 'P22',
 'country': 'United_States',
 'cyanocitta_cristata': 7,
 'date': datetime.datetime(2012, 2, 23, 0, 0),
 'day': 54,
 'effort_area_ha': 0.0,
 'effort_distance_km': 1.931,
 'effort_hrs': 0.83,
 'elev_gt': 182,
 'elev_ned': None,
 'group_id': None,
 'haemorhous_mexicanus': 2,
 'haliaeetus_leucocephalus': 3,
 'housing_density': None,
 'housing_percent_vacant': None,
 'larus_argentatus': 3,
 'larus_delawarensis': 81,
 'loc': {'coordinates': [-83.911171, 43.6727141], 'type': 'Point'},
 'lophodytes_cucullatus': 4,
 'mergus_merganser': 87,
 'month': 2,
 'nlcd2001_fs_c11_7500_pland': 38.0488,
 'nlcd2001_fs_c12_7500_pland': 0.0,
 'nlcd2001_fs_c21_7500_pland': 6.928,
 'nlcd2001_fs_c22

In [9]:
migrations.find_one(filter={'sightings.bird_id': 'zenaida_macroura'},
                    projection=['sightings.$'])

# And then I'll just add the core_covariates to `projection`!!!

{'_id': 'S10000010',
 'sightings': [{'bird_id': 'zenaida_macroura', 'count': 1}]}

In [10]:
# So, I need to read in the list of core covariates.
core_covariates = open('../data/core-covariates.names').readlines()

In [11]:
core_covariates = [cv.split(":")[0].lower() for cv in\
                   open('../data/core-covariates.names').readlines()]

## Planning

We need to decide which bird to use as a demo. The `Species_Analysis_Matrix_V1` document lists species and various properties.

In [12]:
core_covariates

['sampling_event_id',
 'pop00_sqmi',
 'housing_density',
 'housing_percent_vacant',
 'elev_gt',
 'elev_ned',
 'bcr',
 'bailey_ecoregion',
 'omernik_l3_ecoregion',
 'caus_temp_avg',
 'caus_temp_min',
 'caus_temp_max',
 'caus_prec',
 'caus_snow',
 'nlcd2001_fs_c11_7500_pland',
 'nlcd2001_fs_c12_7500_pland',
 'nlcd2001_fs_c21_7500_pland',
 'nlcd2001_fs_c22_7500_pland',
 'nlcd2001_fs_c23_7500_pland',
 'nlcd2001_fs_c24_7500_pland',
 'nlcd2001_fs_c31_7500_pland',
 'nlcd2001_fs_c41_7500_pland',
 'nlcd2001_fs_c42_7500_pland',
 'nlcd2001_fs_c43_7500_pland',
 'nlcd2001_fs_c52_7500_pland',
 'nlcd2001_fs_c71_7500_pland',
 'nlcd2001_fs_c81_7500_pland',
 'nlcd2001_fs_c82_7500_pland',
 'nlcd2001_fs_c90_7500_pland',
 'nlcd2001_fs_c95_7500_pland',
 'nlcd2006_fs_c11_7500_pland',
 'nlcd2006_fs_c12_7500_pland',
 'nlcd2006_fs_c21_7500_pland',
 'nlcd2006_fs_c22_7500_pland',
 'nlcd2006_fs_c23_7500_pland',
 'nlcd2006_fs_c24_7500_pland',
 'nlcd2006_fs_c31_7500_pland',
 'nlcd2006_fs_c41_7500_pland',
 'nlcd2006_

In [13]:
migrations.find_one(filter={'sightings.bird_id' : 'zenaida_macroura'},
                    projection=['sightings.$'] + core_covariates)

{'_id': 'S10000010',
 'bailey_ecoregion': '-222J',
 'bcr': 12,
 'caus_prec': 3,
 'caus_snow': 3,
 'caus_temp_avg': 2,
 'caus_temp_max': 1,
 'caus_temp_min': 2,
 'elev_gt': 182,
 'elev_ned': None,
 'housing_density': None,
 'housing_percent_vacant': None,
 'nlcd2001_fs_c11_7500_pland': 38.0488,
 'nlcd2001_fs_c12_7500_pland': 0.0,
 'nlcd2001_fs_c21_7500_pland': 6.928,
 'nlcd2001_fs_c22_7500_pland': 9.4952,
 'nlcd2001_fs_c23_7500_pland': 4.6392,
 'nlcd2001_fs_c24_7500_pland': 1.5504,
 'nlcd2001_fs_c31_7500_pland': 1.4828,
 'nlcd2001_fs_c41_7500_pland': 4.1796,
 'nlcd2001_fs_c42_7500_pland': 0.1372,
 'nlcd2001_fs_c43_7500_pland': 0.2728,
 'nlcd2001_fs_c52_7500_pland': 0.0636,
 'nlcd2001_fs_c71_7500_pland': 0.8444,
 'nlcd2001_fs_c81_7500_pland': 2.272,
 'nlcd2001_fs_c82_7500_pland': 22.8724,
 'nlcd2001_fs_c90_7500_pland': 5.65,
 'nlcd2001_fs_c95_7500_pland': 1.5636,
 'nlcd2006_fs_c11_7500_pland': 38.3464,
 'nlcd2006_fs_c12_7500_pland': 0.0,
 'nlcd2006_fs_c21_7500_pland': 7.15,
 'nlcd2006_fs

In [14]:
# Alternately, using the flat bird name:
migrations.find_one(filter={'zenaida_macroura' : {'$gt' : 0}},
                   projection=['zenaida_macroura'] + core_covariates)
# Equivalent to
migrations.find_one(filter={'zenaida_macroura' : {'$exists':True}},
                   projection=['zenaida_macroura'] + core_covariates)

{'_id': 'S10000010',
 'bailey_ecoregion': '-222J',
 'bcr': 12,
 'caus_prec': 3,
 'caus_snow': 3,
 'caus_temp_avg': 2,
 'caus_temp_max': 1,
 'caus_temp_min': 2,
 'elev_gt': 182,
 'elev_ned': None,
 'housing_density': None,
 'housing_percent_vacant': None,
 'nlcd2001_fs_c11_7500_pland': 38.0488,
 'nlcd2001_fs_c12_7500_pland': 0.0,
 'nlcd2001_fs_c21_7500_pland': 6.928,
 'nlcd2001_fs_c22_7500_pland': 9.4952,
 'nlcd2001_fs_c23_7500_pland': 4.6392,
 'nlcd2001_fs_c24_7500_pland': 1.5504,
 'nlcd2001_fs_c31_7500_pland': 1.4828,
 'nlcd2001_fs_c41_7500_pland': 4.1796,
 'nlcd2001_fs_c42_7500_pland': 0.1372,
 'nlcd2001_fs_c43_7500_pland': 0.2728,
 'nlcd2001_fs_c52_7500_pland': 0.0636,
 'nlcd2001_fs_c71_7500_pland': 0.8444,
 'nlcd2001_fs_c81_7500_pland': 2.272,
 'nlcd2001_fs_c82_7500_pland': 22.8724,
 'nlcd2001_fs_c90_7500_pland': 5.65,
 'nlcd2001_fs_c95_7500_pland': 1.5636,
 'nlcd2006_fs_c11_7500_pland': 38.3464,
 'nlcd2006_fs_c12_7500_pland': 0.0,
 'nlcd2006_fs_c21_7500_pland': 7.15,
 'nlcd2006_fs

In [15]:
# Getting all of them:
zenaida_macroura = migrations.find(filter={'zenaida_macroura' : {'$exists':True}},
                                   projection=['zenaida_macroura'] + core_covariates)

In [16]:
zenaida_macroura.count()

1542192

In [17]:
projection = dict.fromkeys(['zenaida_macroura'] + core_covariates, 1)

In [18]:
# This object is a pymongo cursor. We need to change it to a DataFrame.
# It's too big to do that so maybe we can sample it.
zenaida_macroura = migrations.aggregate(
    [
        {'$match': {'zenaida_macroura' : {'$exists' : True}}},
        {'$project' : projection},
        {'$sample' : {'size' : 13}}
    ]
)

In [19]:
zenaida_macroura_df = pd.DataFrame(list(zenaida_macroura))

In [20]:
zenaida_macroura_df

,_id,bailey_ecoregion,bcr,caus_prec,caus_snow,caus_temp_avg,caus_temp_max,caus_temp_min,elev_gt,elev_ned,...,nlcd2006_fs_c43_7500_pland,nlcd2006_fs_c52_7500_pland,nlcd2006_fs_c71_7500_pland,nlcd2006_fs_c81_7500_pland,nlcd2006_fs_c82_7500_pland,nlcd2006_fs_c90_7500_pland,nlcd2006_fs_c95_7500_pland,omernik_l3_ecoregion,pop00_sqmi,zenaida_macroura
0,S5007245,-232A,30.0,6,NaN,6,6,7,69.0,78.52,...,0.7272,0.6568,0.3112,2.0432,2.0132,1.7736,0.1592,65,6340.5,1
1,S10629193,-212J,12.0,6,NaN,5,5,5,NaN,NaN,...,12.3052,1.5980,0.7408,11.7180,2.3708,7.4080,0.2916,50,3945.9,7
2,S11566162,-221F,13.0,6,NaN,6,6,7,325.0,306.84,...,0.0404,0.1392,2.2108,13.4804,17.5728,2.6364,0.1248,61,178.0,4
3,S11904484,-313E,16.0,2,1.0,5,6,6,1508.0,1508.36,...,0.0000,20.4220,0.5624,9.3964,0.0500,2.2040,0.0000,22,1373.1,5
4,S9605139,M212A,14.0,5,5.0,1,1,2,243.0,NaN,...,28.9536,1.8308,0.2268,1.3988,0.6584,3.4640,0.8964,58,60.3,3
5,S7416927,-221A,30.0,6,4.0,2,2,2,88.0,NaN,...,33.9084,4.5060,0.6788,6.8988,0.7320,5.8608,0.3936,59,200.0,23
6,S3886855,-232D,NaN,4,1.0,7,7,8,NaN,NaN,...,0.0000,0.1468,0.4256,0.0000,0.0000,1.9900,5.8136,75,NaN,2
7,S6149112,-212F,28.0,5,1.0,3,3,3,263.0,253.42,...,14.9452,0.9360,1.3300,9.5116,1.6488,1.0968,0.7716,60,1237.7,1
8,S2441508,-251C,22.0,6,NaN,6,7,7,NaN,NaN,...,0.5896,0.0672,3.5600,18.9560,8.7164,0.5080,0.0924,40,735.7,8
9,S5354883,-321A,35.0,5,NaN,7,7,8,NaN,NaN,...,0.0000,97.6448,0.0228,0.0000,0.0000,0.0432,0.0404,24,0.6,10


## Read in Core Covariates CSV

So really we'd only want to do this once we've fit the model.

In [21]:
datadir = '/Volumes/Transcend/birt data/eBird raw data'

file = 'srd_point_data_30km_v3.0.csv'
path_to_file = os.path.join(datadir, 'srd_point_data_30km_v3.0.csv')

# Missing values seem to be encoded as "?", so we're going to add this to the na_values argument.
srd30km = pd.read_csv(path_to_file, na_values = '?')

In [22]:
cov_samp = srd30km.sample(13)

In [23]:
# Make sure the covariate sample's names are lowercase.
cov_samp.columns = map(str.lower, cov_samp.columns)

In [24]:
# Joining method one: we remove cols in covariates which aren't in `migrations`.
cov_samp2 = cov_samp.loc[:, cov_samp.columns.isin(core_covariates)]
pd.concat([cov_samp2, zenaida_macroura_df], join = 'outer')

,_id,bailey_ecoregion,bcr,caus_prec,caus_snow,caus_temp_avg,caus_temp_max,caus_temp_min,elev_gt,elev_ned,...,nlcd2006_fs_c43_7500_pland,nlcd2006_fs_c52_7500_pland,nlcd2006_fs_c71_7500_pland,nlcd2006_fs_c81_7500_pland,nlcd2006_fs_c82_7500_pland,nlcd2006_fs_c90_7500_pland,nlcd2006_fs_c95_7500_pland,omernik_l3_ecoregion,pop00_sqmi,zenaida_macroura
67743,NaN,-212F,13.0,NaN,NaN,NaN,NaN,NaN,310.0,310.55,...,7.2500,5.9248,0.7676,31.5032,21.9404,4.3096,0.2968,83.0,49.8,NaN
76056,NaN,-331G,17.0,NaN,NaN,NaN,NaN,NaN,972.0,982.21,...,0.0000,34.0616,63.6876,0.0000,1.0732,0.5096,0.3612,43.0,0.5,NaN
97231,NaN,M221D,28.0,NaN,NaN,NaN,NaN,NaN,91.0,104.22,...,1.5828,0.0000,0.0068,38.7612,2.8116,0.7840,0.0860,66.0,72.9,NaN
103640,NaN,-331G,17.0,NaN,NaN,NaN,NaN,NaN,1218.0,1231.32,...,0.0000,2.5516,49.2472,0.0628,41.9392,2.4564,0.0204,43.0,1.0,NaN
39946,NaN,-222E,24.0,NaN,NaN,NaN,NaN,NaN,211.0,210.74,...,0.3876,0.0380,5.2112,22.7740,1.1776,0.0000,0.0180,71.0,105.4,NaN
8414,NaN,-221H,28.0,NaN,NaN,NaN,NaN,NaN,549.0,542.01,...,6.8400,5.6288,7.4204,6.7540,0.7512,0.3672,0.0000,68.0,46.5,NaN
60517,NaN,-234A,27.0,NaN,NaN,NaN,NaN,NaN,80.0,82.58,...,0.3740,9.2892,0.0716,11.3528,37.8564,6.4500,1.1856,74.0,100.2,NaN
40903,NaN,M331F,16.0,NaN,NaN,NaN,NaN,NaN,1889.0,1900.21,...,0.0212,42.1576,18.5436,0.3608,0.0000,0.5536,0.0204,21.0,5.5,NaN
50351,NaN,-322A,33.0,NaN,NaN,NaN,NaN,NaN,1543.0,1580.99,...,0.0000,99.3008,0.0040,0.0000,0.0000,0.0000,0.0000,14.0,0.1,NaN
94789,NaN,-331H,18.0,NaN,NaN,NaN,NaN,NaN,1158.0,1153.54,...,0.0320,0.1640,68.9228,1.3796,22.1116,0.8940,2.7900,25.0,1.4,NaN


In [25]:
# Joining method two: we add a zero-occurrence column to the covariate DataFrame
# and then inner join.
cov_samp3 = cov_samp
cov_samp3['zenaida_macroura'] = 0
pd.concat([cov_samp3, zenaida_macroura_df], join = 'inner')

,bailey_ecoregion,bcr,elev_gt,elev_ned,housing_density,housing_percent_vacant,nlcd2001_fs_c11_7500_pland,nlcd2001_fs_c12_7500_pland,nlcd2001_fs_c21_7500_pland,nlcd2001_fs_c22_7500_pland,...,nlcd2006_fs_c43_7500_pland,nlcd2006_fs_c52_7500_pland,nlcd2006_fs_c71_7500_pland,nlcd2006_fs_c81_7500_pland,nlcd2006_fs_c82_7500_pland,nlcd2006_fs_c90_7500_pland,nlcd2006_fs_c95_7500_pland,omernik_l3_ecoregion,pop00_sqmi,zenaida_macroura
67743,-212F,13.0,310.0,310.55,19.574237,0.061856,0.1108,0.0,4.5652,0.6076,...,7.2500,5.9248,0.7676,31.5032,21.9404,4.3096,0.2968,83.0,49.8,0
76056,-331G,17.0,972.0,982.21,0.194566,0.169435,0.0148,0.0,0.0000,0.0000,...,0.0000,34.0616,63.6876,0.0000,1.0732,0.5096,0.3612,43.0,0.5,0
97231,M221D,28.0,91.0,104.22,29.660421,0.111111,4.2388,0.0,7.8680,2.3204,...,1.5828,0.0000,0.0068,38.7612,2.8116,0.7840,0.0860,66.0,72.9,0
103640,-331G,17.0,1218.0,1231.32,0.452639,0.146552,0.1460,0.0,0.3980,0.0144,...,0.0000,2.5516,49.2472,0.0628,41.9392,2.4564,0.0204,43.0,1.0,0
39946,-222E,24.0,211.0,210.74,39.576840,0.044566,0.1528,0.0,4.0260,0.5216,...,0.3876,0.0380,5.2112,22.7740,1.1776,0.0000,0.0180,71.0,105.4,0
8414,-221H,28.0,549.0,542.01,20.502175,0.085450,0.2128,0.0,2.6180,0.5684,...,6.8400,5.6288,7.4204,6.7540,0.7512,0.3672,0.0000,68.0,46.5,0
60517,-234A,27.0,80.0,82.58,40.498268,0.076923,0.8364,0.0,6.7872,6.0896,...,0.3740,9.2892,0.0716,11.3528,37.8564,6.4500,1.1856,74.0,100.2,0
40903,M331F,16.0,1889.0,1900.21,3.024832,0.252662,0.0152,0.0,0.0608,0.0024,...,0.0212,42.1576,18.5436,0.3608,0.0000,0.5536,0.0204,21.0,5.5,0
50351,-322A,33.0,1543.0,1580.99,0.110227,0.326923,0.0000,0.0,0.4988,0.1228,...,0.0000,99.3008,0.0040,0.0000,0.0000,0.0000,0.0000,14.0,0.1,0
94789,-331H,18.0,1158.0,1153.54,0.667717,0.180451,0.9756,0.0,1.8836,0.5896,...,0.0320,0.1640,68.9228,1.3796,22.1116,0.8940,2.7900,25.0,1.4,0


Now *that's* the kind of thing we can give to `scikit-learn`.

Outstanding questions:

- Why are some covariates missing from each dataset? Which ones are included in the migrations table and which aren't?
- Is it more correct to include *all* of the migrations table as well as the background covariates, or should I only subsample the observations of a bird and compare that to the background covariates?

I'll research these tomorrow with some lit review.